# View: Revenue by Departments and Providers

## Data Source
- **Departments:** `workspace.hospital_silver.departments`
- **Visits:** `workspace.hospital_silver.visits`
- **providers:** `workspace.hospital_silver.providers`

## Details: 
- Location: `workspace.hospital_gold.view_revenue_departments_providers`
- Description: Sum of Revenue group by departments and providers


In [0]:
# Databricks Storage
catalog_name = "workspace"
schema_bronze = "hospital_bronze"
schema_silver = "hospital_silver"
schema_gold = "hospital_gold"

# view name: name of the view in schema and checkpoint
view_name = "view_revenue_departments_providers"

# data source path
data_source = "s3://buckethospitaldata/view/"

# for streaming: schema and checkpoint location (stored in data source S3 buckets)
checkpoint_location = f"{data_source}_checkpoints/{view_name}"

## Read data from Silver Layer

### Visits (streaming table)

In [0]:
df_visits = spark.read.table(f"{catalog_name}.{schema_silver}.visits")

### Departments (static table)

In [0]:
df_departments = spark.read.table(f"{catalog_name}.{schema_silver}.departments")

### Providers (static table)

In [0]:
df_providers = spark.read.table(f"{catalog_name}.{schema_silver}.providers")

## Join datasets

In [0]:
df_join = df_visits.join(df_departments,df_visits.Department_ID == df_departments.Department_ID,"inner").join(df_providers,df_visits.Provider_ID == df_providers.Provider_ID,"inner")

## Aggregate data

In [0]:
from pyspark.sql.functions import countDistinct,sum,approx_count_distinct
view_revenue_departments_providers  = df_join.groupBy("Department","Provider_Name").agg(sum("Revenue_per_visit").alias("Revenue"),approx_count_distinct("Patient_ID").alias("Number_of_unique_patient")).sort("Department","Provider_Name")
# display(view_revenue_depart_provider)

## Write data as a View in Gold Layer

In [0]:
(
    view_revenue_departments_providers.write
    .format("delta")
    .mode("overwrite")  
    .option("overwriteSchema", "true")  
    .saveAsTable(f"{catalog_name}.{schema_gold}.{view_name}")
)